In [58]:
class MyList[A](val value: A, val next: Option[MyList[A]])

defined class MyList

In [15]:
object MyList // val MyList = new
{
    def apply[A](v: A, nxt: Option[MyList[A]]) =
    new MyList(v,nxt)
}
type YourList[A] = Option[MyList[A]]

val t0 = None
val t1 = Some(new MyList(3, Some(new MyList(4, None))))
// `new` removed
val t2 = Some(MyList(3, Some(MyList(4, None))))

defined object MyList
defined type YourList
t0: None.type = None
t1: Some[MyList[Int]] = Some(ammonite.$sess.cmd1$Helper$MyList@4fb73a20)
t2: Some[MyList[Int]] = Some(ammonite.$sess.cmd1$Helper$MyList@7d54160b)

### Simplification

In [9]:
class MyTree[A](_value: A, _left: Option[MyTree[A]], _right: Option[MyTree[A]]) {
    val value: A = _value
    val left: Option[MyTree[A]] = _left
    val right: Option[MyTree[A]] = _right
}

defined class MyTree

In [10]:
class MyTree[A](value: A, 
                left: Option[MyTree[A]], 
                right: Option[MyTree[A]])

defined class MyTree

In [12]:
type YourTree[A] = Option[MyTree[A]]
val t0 : YourTree[Int] = None
val t1 : YourTree[Int] = Some(new MyTree(3, None, None))
val t2 : YourTree[Int] = Some(new MyTree(3, Some(new MyTree(4, None, None)), None))

defined type YourTree
t0: YourTree[Int] = None
t1: YourTree[Int] = Some(ammonite.$sess.cmd9$Helper$MyTree@4b0b6421)
t2: YourTree[Int] = Some(ammonite.$sess.cmd9$Helper$MyTree@5a9d7a19)

### Nominal Sub Typing a.k.a. Inheritance

`gee_type <: foo_type`

In [29]:
class foo_type(x: Int, y: Int) {
    val a : Int = x
    def b : Int = a + y
    def f(z: Int) : Int = b + y + z
}

class gee_type(x: Int) extends foo_type(x+1, x+2) {
    val c : Int = f(x) + b
    val d = 1
}

defined class foo_type
defined class gee_type

In [28]:
object foo_type {
    def apply(x: Int, y: Int) = new foo_type(x, y)
}
val gee_type = new {
    def apply(x: Int) = new gee_type(x)
}

defined object foo_type
gee_type: {def apply(x: Int): ammonite.$sess.cmd26.instance.gee_type} = ammonite.$sess.cmd27$Helper$$anon$1@711c4618

In [30]:
gee_type(30).c

res29: Int = 188

### Overriding

In [51]:
class gee_type(x: Int) extends foo_type(x+1, x+2) {
    // 외부에서 봤을 땐 바꿔치기
    override def f(z: Int) = super.f(z) * 2 + c
    // 내부에서는 접근 가능
    def g(z: Int) = super.f(z) * 2
    
    val c : Int = f(x) + b
    val d : Int = g(x) + b
}
new gee_type(30).c
new gee_type(30).d

defined class gee_type
res50_1: Int = 313
res50_2: Int = 313

In [52]:
val a: foo_type = new gee_type(30)
a.f(0)

a: foo_type = ammonite.$sess.cmd50$Helper$gee_type@aecd6f4
res51_1: Int = 503

#### danger of overriding
* low readability
* avoid overriding or use interface

In [55]:
class foo_type(x: Int, y: Int) {
    val a : Int = x
    def b : Int = a + y
    def f(z: Int) : Int = b + y + z
}

class gee_type(x: Int) extends foo_type(x+1, x+2) {
    override def b = 10
}

new gee_type(30).f(0)

defined class foo_type
defined class gee_type
res54_2: Int = 42

In [57]:
class foo_type(x: Int, y: Int) {
    val a : Int = x
    def b : Int = 0
    def f(z: Int) : Int = b * z
}

class gee_type(x: Int) extends foo_type(x+1, x+2) {
    override def b = 10
}

// 0 같지만 1000이 나옴
new gee_type(30).f(100)

defined class foo_type
defined class gee_type
res56_2: Int = 1000

#### Example: My List

In [63]:
class MyList[A](value: A, next: Option[MyList[A]])
type YourList[A] = Option[MyList[A]]

val t : YourList[Int] = Some(new MyList(3, Some(new MyList(4, None))))

defined class MyList
defined type YourList
t: YourList[Int] = Some(ammonite.$sess.cmd62$Helper$MyList@6712c65b)

#### 만들기 vs 쓰기

In [64]:
// cannot use
class MyList[A]()
class MyNil[A]() extends MyList[A]
class MyCons[A](val hd: A, val tl: MyList[A]) extends MyList[A]

val t: MyList[Int] = new MyCons(3, new MyCons(4, new MyNil()))

defined class MyList
defined class MyNil
defined class MyCons
t: MyList[Int] = ammonite.$sess.cmd63$Helper$MyCons@faef2ee

In [69]:
class MyList[A]()
case class MyNil[A]() extends MyList[A]
case class MyCons[A](hd: A, tl: MyList[A]) extends MyList[A]

def length(x: MyList[Int]): Int =
    x match {
        case MyNil() => 0
        case MyCons(hd, tl) => 1 + length(tl)
    }

val t: MyList[Int] = MyCons(3, MyCons(4, MyNil()))
length(t)

defined class MyList
defined class MyNil
defined class MyCons
defined function length
t: MyList[Int] = MyCons(3, MyCons(4, MyNil()))
res68_5: Int = 2

#### sealed class

In [70]:
// `sealed` for match case coverage
sealed class MyList[A]()
case class MyNil[A]() extends MyList[A]
case class MyCons[A](hd: A, tl: MyList[A]) extends MyList[A]

def length(x: MyList[Int]): Int =
    x match {
        case MyNil() => 0
        case MyCons(hd, tl) => 1 + length(tl)
    }

val t: MyList[Int] = MyCons(3, MyCons(4, MyNil()))
length(t)

defined class MyList
defined class MyNil
defined class MyCons
defined function length
t: MyList[Int] = MyCons(3, MyCons(4, MyNil()))
res69_5: Int = 2